In [19]:
# Core Deep Learning Libraries
import keras
import numpy as np
import pandas as pd

# MLflow for Experiment Tracking
import mlflow
from mlflow.models import infer_signature

# Hyperparameter Optimization
from hyperopt import (
    STATUS_OK,
    Trials,
    fmin,
    hp,
    tpe
)

# Machine Learning Metrics and Preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [20]:
# Load Wine Quality Dataset
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv", 
    sep=";"
)

In [21]:
# Data Splitting Strategy

## Initial Train-Test Split
train, test = train_test_split(
    data, 
    test_size=0.25,  # 25% for testing
    random_state=42  # Reproducibility
)

## Separate Features and Target Variable for Training Set
train_x = train.drop(['quality'], axis=1).values
train_y = train[['quality']].values.ravel()

## Separate Features and Target Variable for Test Set
test_x = test.drop(['quality'], axis=1).values
test_y = test[['quality']].values.ravel()

## Further Split Training Data into Train and Validation Sets
train_x, valid_x, train_y, valid_y = train_test_split(
    train_x, 
    train_y, 
    test_size=0.20,  # 20% of training data for validation
    random_state=42  # Reproducibility
)

In [22]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    """
    Train an Artificial Neural Network with MLflow tracking
    
    Args:
        params (dict): Hyperparameters for model training
        epochs (int): Number of training epochs
        train_x, train_y: Training data
        valid_x, valid_y: Validation data
        test_x, test_y: Test data
    
    Returns:
        dict: Model evaluation results
    """
    ## Data Normalization
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)

    ## Model Architecture
    model = keras.Sequential([
        keras.Input([train_x.shape[1]]),
        keras.layers.Normalization(mean=mean, variance=var),
        keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(32, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dense(1)
    ])

    ## Model Compilation
    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=params["lr"],
            decay=params.get("decay", 1e-6)
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Early Stopping and Checkpoint
    early_stopping = keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        patience=10, 
        restore_best_weights=True
    )

    model_checkpoint = keras.callbacks.ModelCheckpoint(
        'best_model.h5', 
        save_best_only=True
    )

    ## MLflow Tracking
    with mlflow.start_run(nested=True):
        # Training with callbacks
        history = model.fit(
            train_x, train_y,
            validation_data=(valid_x, valid_y),
            epochs=epochs,
            batch_size=64,
            callbacks=[early_stopping, model_checkpoint],
            verbose=0  # Suppress training logs
        )

    ## Model Evaluation
    eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
    eval_rmse = eval_result[1]

    ## MLflow Logging
    with mlflow.start_run(nested=True):
        # Log hyperparameters
        mlflow.log_params(params)
        
        # Log metrics
        mlflow.log_metric("eval_rmse", eval_rmse)
        mlflow.log_metric("train_loss", history.history['loss'][-1])
        mlflow.log_metric("val_loss", history.history['val_loss'][-1])

        # Log model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

    return {
        "loss": eval_rmse, 
        "status": STATUS_OK, 
        "model": model
    }


In [24]:
def objective(params):
    """
    Objective function for hyperparameter optimization using Hyperopt
    
    Args:
        params (dict): Hyperparameters to be tuned
    
    Returns:
        dict: Results from model training
    """
    try:
        # MLflow will track the parameters and results for each run
        result = train_model(
            params,
            epochs=3,
            train_x=train_x,
            train_y=train_y,
            valid_x=valid_x,  # Corrected from valid_y
            valid_y=valid_y,
            test_x=test_x,
            test_y=test_y
        )
        
        return result
    
    except Exception as e:
        print(f"Error in objective function: {e}")
        return {
            'loss': float('inf'),
            'status': STATUS_FAIL
        }


In [25]:
# Hyperparameter Search Space Configuration
space = {
    # Learning Rate Configuration
    "lr": hp.loguniform(
        "lr", 
        np.log(1e-5),  # Minimum learning rate
        np.log(1e-1)   # Maximum learning rate
    ),
    
    # Momentum Configuration
    "momentum": hp.uniform(
        "momentum", 
        0.0,  # Minimum momentum
        1.0   # Maximum momentum
    )
}

In [27]:
# MLflow Experiment Tracking and Hyperparameter Optimization

try:
    # Set MLflow Experiment
    mlflow.set_experiment("WineQuality")

    # Start MLflow Run
    with mlflow.start_run():
        # Hyperparameter Search Configuration
        trials = Trials()
        best = fmin(
            fn=objective,           # Objective function
            space=space,            # Hyperparameter search space
            algo=tpe.suggest,       # Tree of Parzen Estimators algorithm
            max_evals=4,            # Maximum number of evaluations
            trials=trials
        )

        # Error Handling and Best Run Selection
        try:
            # Sort trials by loss and select best run
            best_run = min(trials.results, key=lambda x: x['loss'])
        except Exception as e:
            print(f"Error selecting best run: {e}")
            best_run = None

        # Logging Best Parameters and Results
        if best_run:
            mlflow.log_params(best)
            mlflow.log_metric("eval_rmse", best_run['loss'])
            
            # Log the best model
            if 'model' in best_run:
                mlflow.tensorflow.log_model(
                    best_run['model'], 
                    "model", 
                    signature=signature
                )

            # Print Results
            print(f"Best Parameters: {best}")
            print(f"Best Evaluation RMSE: {best_run['loss']}")
        else:
            print("No valid runs found during hyperparameter optimization")

except Exception as e:
    print(f"Error in MLflow experiment tracking: {e}")


2025/01/20 15:59:18 INFO mlflow.tracking.fluent: Experiment with name 'WineQuality' does not exist. Creating a new experiment.


  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.6732 - root_mean_squared_error: 0.7845
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8229 - root_mean_squared_error: 0.8739 

 25%|██▌       | 1/4 [00:05<00:16,  5.48s/trial, best loss: 0.8912836313247681]

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 35.0877 - root_mean_squared_error: 5.9183
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 34.4604 - root_mean_squared_error: 5.8650 

 50%|█████     | 2/4 [00:10<00:10,  5.11s/trial, best loss: 0.8912836313247681]

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.6685 - root_mean_squared_error: 1.2652
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.1436 - root_mean_squared_error: 1.4392 

 75%|███████▌  | 3/4 [00:15<00:05,  5.13s/trial, best loss: 0.8912836313247681]

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.6849 - root_mean_squared_error: 0.7837
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7055 - root_mean_squared_error: 0.7966 

100%|██████████| 4/4 [00:20<00:00,  5.03s/trial, best loss: 0.8174808025360107]
Best Parameters: {'lr': np.float64(0.037981348649503636), 'momentum': np.float64(0.44598871471430435)}
Best Evaluation RMSE: 0.8174808025360107


## Validating model with best parameters for Deployments

In [29]:
# Inferencing

from mlflow.models import validate_serving_input

model_uri = 'runs:/e524fcf150544e9a9677772f0cacf7af/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(test_x)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

/Users/shounak/Documents/MLOps/MLFlow/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 930us/step



/Users/shounak/Documents/MLOps/MLFlow/venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


array([[5.749973 ],
       [6.7099586],
       [6.4850783],
       ...,
       [6.34512  ],
       [6.921927 ],
       [5.852235 ]], dtype=float32)

In [30]:
# Load model as PyFuncModel.
model_uri = 'runs:/e524fcf150544e9a9677772f0cacf7af/model'
loaded_model = mlflow.pyfunc.load_model(model_uri)

# Predict on a Pandas DataFrame
loaded_model.predict(pd.DataFrame(test_x))

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 925us/step


/Users/shounak/Documents/MLOps/MLFlow/venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


array([[5.749973 ],
       [6.7099586],
       [6.4850783],
       ...,
       [6.34512  ],
       [6.921927 ],
       [5.852235 ]], dtype=float32)

In [ ]:
# Alternative: Register the model through code 
#mlflow.register_model(model_uri, "wine-quality")